# Ejercicio 4 - Dimensionalidad
Genere datasets con C = 0.78, n = 250 para el conjunto de entrenamiento y n = 10000 para el de test, variando esta vez el valor de d según la siguiente lista: 2, 4, 8, 16, 32. Para cada valor de d cree 20 conjuntos distintos de entrenamiento, y uno solo de test. Genere una gráfica del train y test error en función de d para el problema "paralelo" y el "diagonal" (todos en la misma gráfica). Discuta los resultados.

## Definición de funciones
Comencemos por definir las funciones que nos serviran para el caso de diagonales y el caso de paralelas.

In [4]:
from Generadores import diagonales, paralelas
from Comunes import standardTree
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import zero_one_loss

In [6]:
# Definimos una funcion que dados los parametros y la lista de valores de d (dimensionalidad) genere la lista de training sets
def generateTrainingSetsD(setCount, generator, d, n, dims):

    # Inicializamos la lista de conjuntos de entrenamiento
    trainingSets = [[] for _ in range(len(dims))]
    
    # Para cada valor de C
    for i, d in enumerate(dims):

        # Generamos setCount conjuntos de entrenamiento de tamaño n
        for _ in range(setCount):
            trainingSets[i].append(generator(n, d, C))
            # time.sleep(1)    

    return trainingSets

In [ ]:
# La funcion para generar clasificadores es identica a la del ejercicio anterior, donde setSizesCount es cValsCount
def generateClassifiers(dimsCount, setCount, trainingSets):
    # Creamos la lista de listas de clasificadores. La lista contiene setSizesCount listas con setCount clasificadores cada una.
    classifiers = [[] for _ in range(cValsCount)]
    
    # Creamos los clasificadores
    for classifierList in classifiers:
        for i in range(setCount):
            classifierList.append(standardTree())
    
    # Entrenamos los clasificadores con su set de entrenamiento correspondiente.
    for i,classifierList in enumerate(classifiers):
        for j,classifier in enumerate(classifierList):
            classifier.fit(list(trainingSets[i][j]["Input"]), list(trainingSets[i][j]["Output"]))

    return classifiers

## Definición de parámetros y clasificadores
Pasemos ahora a utilizar las funciones previamente definidas para generar los datos necesarios para entrenar a los distintos clasificadores.

In [1]:
# Definimos los parametros
dims = [2, 4, 8, 16, 32]
ntrain = 250
ntest = 10000
cval = 0.78

setCount = 20
dimCount = 5